# Problem 1

We want to implement P1-FEM in 1D. For benchmarking, we use a simple Poisson problem in 1D:
$$
    \begin{cases}
        -u''=f\quad&\text{in }\Omega=(-1,1),\\
        u(-1)=u_0(-1),\quad&\\
        u'(1)=g(1),\quad&\\
    \end{cases}
$$
with $f(x)=9\sin(3x+3)$, $u_0(-1)=0$, and $g(1)=3\cos(6)$. The exact solution is $u(x)=\sin(3x+3)$.

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import sympy as sp

## (a)

Derive a weak (Galerkin) formulation for the problem.

* $V=...$
* $a(u,v)=...$
* $b(v)=...$

## (b)

Implement a function that computes the four mesh-related quantities `grid`, `elements`, `endpoints`, and `boundaries`.

In [2]:
def createmesh1D(h):
    """
    Function to create mesh stuff with mesh-size h for P1-FEM.
    """
    ...
    return grid, elements, endpoints, boundaries

In [3]:
# check your implementation of createmesh1D
from pickle import load
data = createmesh1D(0.1)
with open("check/createmesh1D_k1.pkl", "rb") as f: data_ex = load(f)
for dat, dat_ex, name in zip(data, data_ex,
                             ["grid", "elements", "endpoints", "boundaries"]):
    print(f"'{name}' should be {dat_ex}\ncomputed '{name}' is {dat}\n")

NameError: name 'grid' is not defined

## (c)

Implement a function that assembles the P1-FEM LHS matrix.

In [ ]:
def assemble_LHS_poisson(mesh):
    """
    Function to assemble P1-FEM LHS.
    """
    grid, elements, endpoints, boundaries = mesh
    ...
    mat = np.zeros((len(grid), len(grid)))
    for el in elements: # volume terms (must integrate)
        ...
    for j, b in zip(endpoints, boundaries): # Dirichlet and Robin BCs
        # j is index of DoF, b is boundary flag
        ...
    return mat

In [ ]:
# check your implementation of assemble_LHS_poisson
from pickle import load
mat = assemble_LHS_poisson(createmesh1D(0.1))
with open("check/assemble_LHS_poisson.pkl", "rb") as f: mat_ex = load(f)
print(f"stiffness matrix should be {mat_ex}\ncomputed stiffness matrix is {mat}")

## (d)

Write a function that assembles the P1-FEM RHS vector.

In [ ]:
def quadrature1D(domain, integrand):
    """
    Function to perform quadrature of callable function integrand over interval specified by domain.
        \int_{domain} integrand(x) dx
    """
    ...

def assemble_RHS_poisson(mesh, u0, f, g):
    """
    Function to assemble P1-FEM RHS.
    """
    grid, elements, endpoints, boundaries = mesh
    ...
    vec = np.zeros(len(grid))
    for el in elements: # volume terms (must integrate)
        ...
    for j, b in zip(endpoints, boundaries): # all BCs
        # j is index of DoF, b is boundary flag
        ...
    return vec

In [ ]:
# check your implementation of quadrature1D
from pickle import load
val = quadrature1D([0., 1.], lambda x: 4 * x ** 3)
print(f"integral value is 1.\nuser-approximated value is {val}")
print("NOTE: results may vary slightly depending on the implemented quadrature formula")

In [ ]:
# check your implementation of assemble_RHS_poisson
from pickle import load
vec = assemble_RHS_poisson(createmesh1D(0.1), lambda x: 0.,
                           lambda x: 9 * np.sin(3 * x + 3),
                           lambda x: 3 * np.cos(6))
with open("check/assemble_RHS_poisson.pkl", "rb") as f: vec_ex = load(f)
print(f"RHS vector should be {vec_ex}\ncomputed RHS vector is {vec}")
print("NOTE: results may vary slightly depending on the implemented quadrature formula")

## (e)

Compute and plot the P1-FE solution.

In [ ]:
def solve_poisson(h):
    """
    Function to compute the P1-FEM solution uh with mesh-size h.
    """
    ...
...

Compute the FE error in the $L^\infty(\Omega)$ norm

In [ ]:
def eval_uh(mesh, uh, x):
    """
    Function to evaluate the P1-FEM solution uh (based on mesh) at new point(s) x.
    """
    ...
...

Compute the FE error in the $L^2(\Omega)$- and $H^1(\Omega)$-norms.

In [ ]:
def eval_uh_prime(mesh, uh, x):
    """
    Function to evaluate the first derivative of the P1-FEM solution uh (based on mesh) at new point(s) x.
    """
    ...
...

## (f)

Solve the problem on a sequence of finer and finer meshes. Identify the convergence rates of the errors.

In [ ]:
...
hs = .5 ** np.arange(7)
for h in hs:
    ...
...

## (g)

Solve the problem on a sequence of finer and finer meshes. Identify the convergence rates of the FEM matrix's condition number.

In [ ]:
...
hs = .5 ** np.arange(7)
for h in hs:
    ...
...

## (h)

Derive two weak formulations for a modified problem with non-homogeneous Dirichlet BCs.

1) Petrov-Galerkin
    ...

2) Lifting.
    ...

Solve the non-homogeneous problem and study the convergence rates.

In [ ]:
def solve_poisson_nonhomogeneous(h):
    """
    Function to compute the P1-FEM solution uh of the problem with non-homogeneous Dirichlet BCs with mesh-size h.
    """
    ...
...
hs = .5 ** np.arange(7)
for h in hs:
    ...
...

## (i)

Derive a weak formulation for a modified problem with a Robin BC. Study the convergence rates.

* $a(u,v)=...$

In [ ]:
def solve_poisson_robin(h):
    """
    Function to compute the P1-FEM solution uh of the problem with Robin BCs with mesh-size h.
    """
    ...
...
hs = .5 ** np.arange(7)
for h in hs:
    ...
...